In [171]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd

urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))



Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [183]:
#Process the dataset for movies, users,ratings and genre
# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')


ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')


# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

# Since the ids start at 1, we shift them to start at 0.
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: str(x-1))
ratings["user_id"] = ratings["user_id"].apply(lambda x: str(x-1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))



In [201]:
#Get all the genres for a movie
import numpy as np
genre_occurences = movies[genre_cols].sum().to_dict()

genres_encoded = {x: i for i, x in enumerate(genre_cols)}
# print(genre_occurences)
# print(genres_encoded)
# print(genre_cols, movies)

def get_genres(movies, genres):
    def get_all_genres(gs):
        active = [str(genres_encoded[genre]) for genre, g in zip(genres, gs) if g==1]
        if len(active) == 0:
            return '0'
        return ','.join((active))
    movies['all_genres'] = [
        get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]
print(genre_cols)
print('test', movies['Action'])    
get_genres(movies, genre_cols)
print(movies['all_genres'])

['genre_unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
test 0       0
1       1
2       0
3       1
4       0
       ..
1677    0
1678    0
1679    0
1680    0
1681    0
Name: Action, Length: 1682, dtype: int64
0        3,4,5
1       1,2,16
2           16
3        1,5,8
4       6,8,16
         ...  
1677         8
1678     14,16
1679      8,14
1680         5
1681         8
Name: all_genres, Length: 1682, dtype: object


In [202]:
movies.head(3)

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,all_genres
0,0,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1995,"3,4,5"
1,1,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,1,0,0,1995,"1,2,16"
2,2,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1995,16


In [207]:
rating_details_sample = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

In [9]:
rating_details_sample.shape

(100000, 33)

In [10]:
rating_details_sample.head(10)

,user_id,movie_id,rating,unix_timestamp,title,release_date,video_release_date,imdb_url,genre_unknown,Action,...,Sci-Fi,Thriller,War,Western,year,all_genres,age,sex,occupation,zip_code
0,195,241,3.0,881250949,Kolya (1996),24-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Kolya%20(1996),0,0,...,0,0,0,0,1997,5,49,M,writer,55105
1,195,256,2.0,881251577,Men in Black (1997),04-Jul-1997,NaN,http://us.imdb.com/M/title-exact?Men+in+Black+...,0,1,...,1,0,0,0,1997,"1,2,5,15",49,M,writer,55105
2,195,110,4.0,881251793,"Truth About Cats & Dogs, The (1996)",26-Apr-1996,NaN,http://us.imdb.com/M/title-exact?Truth%20About...,0,0,...,0,0,0,0,1996,"5,14",49,M,writer,55105
3,195,24,4.0,881251955,"Birdcage, The (1996)",08-Mar-1996,NaN,"http://us.imdb.com/M/title-exact?Birdcage,%20T...",0,0,...,0,0,0,0,1996,5,49,M,writer,55105
4,195,381,4.0,881251843,"Adventures of Priscilla, Queen of the Desert, ...",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Adventures%20...,0,0,...,0,0,0,0,1994,"5,8",49,M,writer,55105
5,195,201,3.0,881251728,Groundhog Day (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Groundhog%20D...,0,0,...,0,0,0,0,1993,"5,14",49,M,writer,55105
6,195,152,5.0,881251820,"Fish Called Wanda, A (1988)",01-Jan-1988,NaN,http://us.imdb.com/M/title-exact?Fish%20Called...,0,0,...,0,0,0,0,1988,5,49,M,writer,55105
7,195,285,5.0,881250949,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,...,0,0,1,0,1996,"8,14,17",49,M,writer,55105
8,195,65,3.0,881251911,While You Were Sleeping (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?While%20You%2...,0,0,...,0,0,0,0,1995,"5,14",49,M,writer,55105
9,195,844,4.0,881251954,That Thing You Do! (1996),28-Sep-1996,NaN,http://us.imdb.com/M/title-exact?That%20Thing%...,0,0,...,0,0,0,0,1996,5,49,M,writer,55105


In [219]:
rating_details_sample['user_id']=rating_details_sample['user_id'].astype(int)
rating_details_sample['movie_id']=rating_details_sample['movie_id'].astype(int)

In [220]:
rating_details_sample=rating_details_sample.set_index(['user_id','unix_timestamp']).sort_index()

In [221]:
rating_details_sample =rating_details_sample.reset_index()


In [225]:
rating_details_sample.head(100)

,user_id,unix_timestamp,level_0,index,movie_id,rating,title,release_date,video_release_date,imdb_url,...,Sci-Fi,Thriller,War,Western,year,all_genres,age,sex,occupation,zip_code
0,0,874965478,1544,1544,171,5.0,"Empire Strikes Back, The (1980)",01-Jan-1980,NaN,http://us.imdb.com/M/title-exact?Empire%20Stri...,...,1,0,1,0,1980,"1,2,8,14,15,17",24,M,technician,85711
1,0,874965478,1623,1623,167,5.0,Monty Python and the Holy Grail (1974),01-Jan-1974,NaN,http://us.imdb.com/M/title-exact?Monty%20Pytho...,...,0,0,0,0,1974,5,24,M,technician,85711
2,0,874965518,1510,1510,164,5.0,Jean de Florette (1986),01-Jan-1986,NaN,http://us.imdb.com/M/title-exact?Jean%20de%20F...,...,0,0,0,0,1986,8,24,M,technician,85711
3,0,874965556,1617,1617,155,4.0,Reservoir Dogs (1992),01-Jan-1992,NaN,http://us.imdb.com/M/title-exact?Reservoir%20D...,...,0,1,0,0,1992,"6,16",24,M,technician,85711
4,0,874965677,1503,1503,195,5.0,Dead Poets Society (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Dead%20Poets%...,...,0,0,0,0,1989,8,24,M,technician,85711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,875072956,1584,1584,190,5.0,Amadeus (1984),01-Jan-1984,NaN,http://us.imdb.com/M/title-exact?Amadeus%20(1984),...,0,0,0,0,1984,"8,13",24,M,technician,85711
96,0,875072956,1590,1590,196,5.0,"Graduate, The (1967)",01-Jan-1967,NaN,"http://us.imdb.com/M/title-exact?Graduate,%20T...",...,0,0,0,0,1967,"8,14",24,M,technician,85711
97,0,875072956,1622,1622,93,2.0,Home Alone (1990),01-Jan-1990,NaN,http://us.imdb.com/M/title-exact?Home%20Alone%...,...,0,0,0,0,1990,"4,5",24,M,technician,85711
98,0,875072956,1625,1625,183,4.0,Army of Darkness (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Army%20of%20D...,...,1,0,0,0,1993,"1,2,5,11,15",24,M,technician,85711


In [234]:
# Get the like and dislike movie list
import numpy as np
rating_details_sample['movie_type'] = np.where(rating_details_sample['rating'] >=3, 'like','dislike')
rating_details_sample['movie_name'] = rating_details_sample['title'].str[:-6]

In [235]:
rating_details_sample.head(10)

,user_id,unix_timestamp,level_0,index,movie_id,rating,title,release_date,video_release_date,imdb_url,...,War,Western,year,all_genres,age,sex,occupation,zip_code,movie_type,movie_name
0,0,874965478,1544,1544,171,5.0,"Empire Strikes Back, The (1980)",01-Jan-1980,NaN,http://us.imdb.com/M/title-exact?Empire%20Stri...,...,1,0,1980,"1,2,8,14,15,17",24,M,technician,85711,like,"Empire Strikes Back, The"
1,0,874965478,1623,1623,167,5.0,Monty Python and the Holy Grail (1974),01-Jan-1974,NaN,http://us.imdb.com/M/title-exact?Monty%20Pytho...,...,0,0,1974,5,24,M,technician,85711,like,Monty Python and the Holy Grail
2,0,874965518,1510,1510,164,5.0,Jean de Florette (1986),01-Jan-1986,NaN,http://us.imdb.com/M/title-exact?Jean%20de%20F...,...,0,0,1986,8,24,M,technician,85711,like,Jean de Florette
3,0,874965556,1617,1617,155,4.0,Reservoir Dogs (1992),01-Jan-1992,NaN,http://us.imdb.com/M/title-exact?Reservoir%20D...,...,0,0,1992,"6,16",24,M,technician,85711,like,Reservoir Dogs
4,0,874965677,1503,1503,195,5.0,Dead Poets Society (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Dead%20Poets%...,...,0,0,1989,8,24,M,technician,85711,like,Dead Poets Society
5,0,874965677,1690,1690,165,5.0,Manon of the Spring (Manon des sources) (1986),01-Jan-1986,NaN,http://us.imdb.com/M/title-exact?Manon%20des%2...,...,0,0,1986,8,24,M,technician,85711,like,Manon of the Spring (Manon des sources)
6,0,874965678,1620,1620,186,4.0,"Godfather: Part II, The (1974)",01-Jan-1974,NaN,http://us.imdb.com/M/title-exact?Godfather:%20...,...,0,0,1974,"1,6,8",24,M,technician,85711,like,"Godfather: Part II, The"
7,0,874965706,1534,1534,13,5.0,"Postino, Il (1994)",01-Jan-1994,NaN,"http://us.imdb.com/M/title-exact?Postino,%20Il...",...,0,0,1994,"8,14",24,M,technician,85711,like,"Postino, Il"
8,0,874965706,1556,1556,126,5.0,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",...,0,0,1972,"1,6,8",24,M,technician,85711,like,"Godfather, The"
9,0,874965706,1599,1599,249,4.0,"Fifth Element, The (1997)",09-May-1997,NaN,http://us.imdb.com/M/title-exact?Fifth%20Eleme...,...,0,0,1997,"1,15",24,M,technician,85711,like,"Fifth Element, The"


In [242]:
user_ids = rating_details_sample["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

movie_ids = rating_details_sample["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}

title_ids = rating_details_sample["movie_name"].unique().tolist()
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

rating_details_sample["user"] = rating_details_sample["user_id"].map(user2user_encoded)
rating_details_sample["movie"] = rating_details_sample["movie_id"].map(movie2movie_encoded)
rating_details_sample["title_d"] = rating_details_sample["movie_name"].map(title2title_encoded)


In [243]:
rating_details_sample.head(5)

,user_id,unix_timestamp,level_0,index,movie_id,rating,title,release_date,video_release_date,imdb_url,...,all_genres,age,sex,occupation,zip_code,movie_type,movie_name,user,movie,title_d
0,0,874965478,1544,1544,171,5.0,"Empire Strikes Back, The (1980)",01-Jan-1980,NaN,http://us.imdb.com/M/title-exact?Empire%20Stri...,...,"1,2,8,14,15,17",24,M,technician,85711,like,"Empire Strikes Back, The",0,0,0
1,0,874965478,1623,1623,167,5.0,Monty Python and the Holy Grail (1974),01-Jan-1974,NaN,http://us.imdb.com/M/title-exact?Monty%20Pytho...,...,5,24,M,technician,85711,like,Monty Python and the Holy Grail,0,1,1
2,0,874965518,1510,1510,164,5.0,Jean de Florette (1986),01-Jan-1986,NaN,http://us.imdb.com/M/title-exact?Jean%20de%20F...,...,8,24,M,technician,85711,like,Jean de Florette,0,2,2
3,0,874965556,1617,1617,155,4.0,Reservoir Dogs (1992),01-Jan-1992,NaN,http://us.imdb.com/M/title-exact?Reservoir%20D...,...,"6,16",24,M,technician,85711,like,Reservoir Dogs,0,3,3
4,0,874965677,1503,1503,195,5.0,Dead Poets Society (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Dead%20Poets%...,...,8,24,M,technician,85711,like,Dead Poets Society,0,4,4


In [246]:
sample_data=rating_details_sample[['user','occupation','sex']]

In [248]:
sample_data=sample_data.reset_index()

In [249]:
sample_data

,level_0,index,user,occupation,sex
0,0,0,0,technician,M
1,1,1,0,technician,M
2,2,2,0,technician,M
3,3,3,0,technician,M
4,4,4,0,technician,M
...,...,...,...,...,...
99995,99995,99995,942,student,M
99996,99996,99996,942,student,M
99997,99997,99997,942,student,M
99998,99998,99998,942,student,M


In [19]:
rating_details_sample["movie"].max()

1681

In [251]:
movie_list = rating_details_sample.groupby(['user','movie_type'])['movie'].apply(list).reset_index()
title_list = rating_details_sample.groupby(['user'])['title_d'].apply(list).reset_index()
genre_list = rating_details_sample.groupby(['user'])['all_genres'].unique().apply(list).reset_index()

In [254]:
genre_list

,user,all_genres
0,0,"[1,2,8,14,15,17, 5, 8, 6,16, 1,6,8, 8,14, 1,15..."
1,1,"[8,14,17, 8,15, 8, 6,11,13,16, 11,16, 5, 5,6,8..."
2,2,"[6,10,13,16, 8, 13,16, 1,16, 8,14, 5,8,16, 5,1..."
3,3,"[8,15, 11,16, 1,16, 1,13,14,16, 1,2,15,17, 6,8..."
4,4,"[0, 1,2,15, 1, 1,15,17, 1,2,13, 1,2,5,15, 1,15..."
...,...,...
938,938,"[1,2, 1,8,17, 1,16, 8,15, 1,6,8, 8, 1,2,5,15, ..."
939,939,"[8,14,17, 5,12,14, 8, 5, 5,8, 15,16, 1,2,15,17..."
940,940,"[8,15, 1,16, 5, 1,2,15,16, 1,2,16, 3,5,16, 1,2..."
941,941,"[8, 8,16, 5,8, 5, 1,8,14, 1,5,12, 8,15, 2,4, 1..."


In [21]:
# Get the unique set of genre for all the users
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x: list(set(','.join(x))) ) 
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x:[ x for x in x if x.isdigit() ])

In [22]:
user_video_list = movie_list.pivot(index='user', columns='movie_type', values='movie').reset_index()

In [23]:
user_video_list.fillna(rating_details_sample["movie"].max()+1, inplace=True)

In [24]:
sample_data = sample_data.drop('index',axis=1)

In [25]:
sample_data =sample_data.drop_duplicates()

In [26]:
user_final_list =pd.merge(user_video_list,title_list, how= 'left')
user_title_list1 = pd.merge(user_final_list,genre_list, how='left')
user_title_list = pd.merge(user_title_list1,sample_data, how='left')

In [27]:
user_title_list1.head(3)

,user,dislike,like,title_d,all_genres
0,0,"[31, 32, 33, 35, 36, 55, 71, 81, 97, 99, 107, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 5, 2, 9, 4, 1, 6, 8, 7, 3]"
1,1,"[279, 298, 130, 313, 314]","[272, 250, 273, 274, 275, 276, 277, 278, 280, ...","[271, 249, 272, 273, 274, 275, 276, 277, 278, ...","[0, 5, 2, 9, 4, 1, 6, 8, 7, 3]"
2,2,"[302, 317, 309, 318, 275, 250, 321, 253, 322, ...","[316, 125, 278, 319, 320, 324, 325, 326, 328, ...","[300, 314, 315, 307, 14, 277, 316, 317, 318, 2...","[0, 5, 2, 4, 1, 8, 6, 7, 3]"


In [28]:
user_title_list['like'] =user_title_list['like'].apply(lambda x: x if type(x) is list else [x])
user_title_list['dislike'] =user_title_list['dislike'].apply(lambda x: x if type(x) is list else [x])

In [29]:
user_title_list['predict_labels'] = user_title_list['like'].apply(lambda x: (x[-1]))

In [30]:
user_title_list['like']=user_title_list['like'].apply(lambda x: (x[:-1]))

In [61]:
pd.DataFrame(user_title_list[['user','dislike','like','title_d','all_genres','predict_labels']]).head(4)

,user,dislike,like,title_d,all_genres,predict_labels
0,0,"[31, 32, 33, 35, 36, 55, 71, 81, 97, 99, 107, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 5, 2, 9, 4, 1, 6, 8, 7, 3]",269
1,1,"[279, 298, 130, 313, 314]","[272, 250, 273, 274, 275, 276, 277, 278, 280, ...","[271, 249, 272, 273, 274, 275, 276, 277, 278, ...","[0, 5, 2, 9, 4, 1, 6, 8, 7, 3]",315
2,2,"[302, 317, 309, 318, 275, 250, 321, 253, 322, ...","[316, 125, 278, 319, 320, 324, 325, 326, 328, ...","[300, 314, 315, 307, 14, 277, 316, 317, 318, 2...","[0, 5, 2, 4, 1, 8, 6, 7, 3]",354
3,3,[361],"[250, 275, 309, 345, 254, 344, 355, 278, 350, ...","[249, 274, 307, 343, 253, 342, 352, 277, 348, ...","[5, 2, 4, 1, 8, 6, 7, 3]",55


In [32]:
user_title_list_e=user_title_list[(user_title_list.user >= 1)&
                                  (user_title_list.user <= 500)]

In [33]:
user_title_list.shape

(943, 8)

In [130]:
EMBEDDING_DIMS = 16
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES=rating_details_sample["movie"].max()+2

LEARNING_RATE = 0.003

In [131]:
import tensorflow as tf
class MaskedEmbeddingsAggregatorLayer(tf.keras.layers.Layer):
    def __init__(self, agg_mode='sum', **kwargs):
        super(MaskedEmbeddingsAggregatorLayer, self).__init__(**kwargs)

        if agg_mode not in ['sum', 'mean']:
            raise NotImplementedError('mode {} not implemented!'.format(agg_mode))
        self.agg_mode = agg_mode
    
    @tf.function
    def call(self, inputs, mask=None):
        masked_embeddings = tf.ragged.boolean_mask(inputs, mask)
        if self.agg_mode == 'sum':
            aggregated =  tf.reduce_sum(masked_embeddings, axis=1)
        elif self.agg_mode == 'mean':
            aggregated = tf.reduce_mean(masked_embeddings, axis=1)
        
        return aggregated
    
    def get_config(self):
        # this is used when loading a saved model that uses a custom layer
        return {'agg_mode': self.agg_mode}

In [132]:
class L2NormLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormLayer, self).__init__(**kwargs)
    
    @tf.function
    def call(self, inputs, mask=None):
        if mask is not None:
            inputs = tf.ragged.boolean_mask(inputs, mask).to_tensor()
        return tf.math.l2_normalize(inputs, axis=-1)

    def compute_mask(self, inputs, mask):
        return mask

In [157]:
#---inputs
import tensorflow as tf
import datetime
import os
input_title = tf.keras.Input(shape=(None, ), name='movie_name')
inp_video_liked = tf.keras.layers.Input(shape=(None,), name='like')
inp_video_disliked = tf.keras.layers.Input(shape=(None,), name='dislike')
input_genre = tf.keras.Input(shape=(None, ), name='genre')


#--- layers
features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=DENSE_UNITS, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax, name='dense_output')

#--- features
features_embeddings = features_embedding_layer(input_title)
l2_norm_features = l2_norm_1(features_embeddings)
avg_features = avg_embeddings(l2_norm_features)

labels_liked_embeddings = labels_embedding_layer(inp_video_liked)
l2_norm_liked = l2_norm_1(labels_liked_embeddings)
avg_liked = avg_embeddings(l2_norm_liked)

labels_disliked_embeddings = labels_embedding_layer(inp_video_disliked)
l2_norm_disliked = l2_norm_1(labels_disliked_embeddings)
avg_disliked = avg_embeddings(l2_norm_disliked)

labels_genre_embeddings = labels_embedding_layer(input_genre)
l2_norm_genre = l2_norm_1(labels_genre_embeddings)
avg_genre = avg_embeddings(l2_norm_genre)



concat_inputs = tf.keras.layers.Concatenate(axis=1)([avg_features,
                                                     avg_liked,
                                                     avg_disliked,
                                                     avg_genre
                                                     ])
# Dense Layers

dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
#dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[input_title, inp_video_liked, 
            inp_video_disliked
            ,input_genre  
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy')

In [162]:
model.summary()


Model: "functional_41"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_name (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
like (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
dislike (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
genre (InputLayer)              [(None, None)]       0                                            
______________________________________________________________________________________

In [166]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True,dpi=96)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [160]:
model.fit([tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['title_d']),
           tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['like']),
           tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['dislike']),
            tf.keras.preprocessing.sequence.pad_sequences(user_title_list_e['all_genres'])
           ],user_title_list_e['predict_labels'].values,callbacks=[tensorboard_callback],
           steps_per_epoch=1, epochs=10,verbose=1)

Epoch 1/10


InvalidArgumentError:  indices[3828] = 4501 is not in [0, 4500)
	 [[{{node functional_41/aggregate_embeddings/PartitionedCall_3/RaggedMask/boolean_mask/GatherV2}}]] [Op:__inference_train_function_67559]

Function call stack:
train_function
